# 1. LAST-FM: Baseline ALS Recommender

This notebook implements a simple ALS recommender based on the LastFM user listening dataset. It uses spark and is written in Scala. Minimal data cleaning/pre-processing is performed to provide a baseline model. 

## 1.1 Imports and set up 

Key libraries are imported, the spark session is initialised and the listening data is loaded in. 

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.0.0` // Or use any other 2.x version here

import $ivy.$                                   // Or use any other 2.x version here

In [2]:
import $ivy.`org.apache.spark::spark-mllib:3.0.0`

import $ivy.$                                    

In [3]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.feature.{StringIndexer,IndexToString}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.recommendation.ALS
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.mllib.evaluation.{RankingMetrics, RegressionMetrics}
import org.apache.spark.ml.feature.RobustScaler
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.functions.vector_to_array
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql._

import org.apache.spark.sql.SparkSession

import org.apache.spark.ml.feature.{StringIndexer,IndexToString}

import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.recommendation.ALS

import org.apache.spark.ml.evaluation.RegressionEvaluator

import org.apache.spark.mllib.evaluation.{RankingMetrics, RegressionMetrics}

import org.apache.spark.ml.feature.RobustScaler

import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.functions.vector_to_array

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql._

In [4]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)



val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .appName("lastfm")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.log4j.{Level, Logger}

spark: SparkSession = org.apache.spark.sql.SparkSession@355a2e2c

In [5]:
// path to last-fm dataset. Can be downloaded here: http://millionsongdataset.com/lastfm/
var data_path:String = "../resources/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv"

data_path: String = "../resources/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv"

In [6]:
// schema defined below to set column names and types. 
val schema = new StructType()
            .add("user_id", StringType, true)
            .add("timestamp", StringType, true)
            .add("artist_id", StringType, true)
            .add("artist_name", StringType, true)
            .add("track_id", StringType, true)
            .add("track_name", StringType, true)

schema: StructType = StructType(
  StructField("user_id", StringType, true, {}),
  StructField("timestamp", StringType, true, {}),
  StructField("artist_id", StringType, true, {}),
  StructField("artist_name", StringType, true, {}),
  StructField("track_id", StringType, true, {}),
  StructField("track_name", StringType, true, {})
)

In [7]:
// read in data
val listener_data = spark.read.option("header", false).schema(schema).format("org.apache.spark.sql.execution.datasources.csv.CSVFileFormat").option("sep", "\t").load(data_path)
listener_data.show()

show at cmd6.sc:2

1 / 1

+-----------+--------------------+--------------------+---------------+--------------------+--------------------+
|    user_id|           timestamp|           artist_id|    artist_name|            track_id|          track_name|
+-----------+--------------------+--------------------+---------------+--------------------+--------------------+
|user_000001|2009-05-04T23:08:57Z|f1b1cf71-bd35-4e9...|      Deep Dish|                null|Fuck Me Im Famous...|
|user_000001|2009-05-04T13:54:10Z|a7f7df4a-77d8-4f1...|       坂本龍一|                null|Composition 0919 ...|
|user_000001|2009-05-04T13:52:04Z|a7f7df4a-77d8-4f1...|       坂本龍一|                null|Mc2 (Live_2009_4_15)|
|user_000001|2009-05-04T13:42:52Z|a7f7df4a-77d8-4f1...|       坂本龍一|                null|Hibari (Live_2009...|
|user_000001|2009-05-04T13:42:11Z|a7f7df4a-77d8-4f1...|       坂本龍一|                null|Mc1 (Live_2009_4_15)|
|user_000001|2009-05-04T13:38:31Z|a7f7df4a-77d8-4f1...|       坂本龍一|                null|To Stanford (Liv

listener_data: DataFrame = [user_id: string, timestamp: string ... 4 more fields]

We remove all NaN values as determining ratings with missing data would be problematic for a simple model. Additionally, the timestamp column is also dropped as its not directly contributing to the user-item matrix we are trying to build. 

In [8]:
val df = listener_data.drop("timestamp").na.drop()
df.show()

show at cmd7.sc:2

1 / 1

+-----------+--------------------+----------------+--------------------+--------------------+
|    user_id|           artist_id|     artist_name|            track_id|          track_name|
+-----------+--------------------+----------------+--------------------+--------------------+
|user_000001|a7f7df4a-77d8-4f1...|        坂本龍一|f7c1f8f8-b935-45e...|The Last Emperor ...|
|user_000001|a7f7df4a-77d8-4f1...|        坂本龍一|475d4e50-cebb-4cd...|Tibetan Dance (Ve...|
|user_000001|ba2f4f3b-0293-4bc...|      Underworld|dc394163-2b78-4b5...|Boy, Boy, Boy (Sw...|
|user_000001|ba2f4f3b-0293-4bc...|      Underworld|340d9a0b-9a43-409...|Crocodile (Innerv...|
|user_000001|a16e47f5-aa54-47f...| Ennio Morricone|0b04407b-f517-4e0...|Ninna Nanna In Bl...|
|user_000001|463a94f1-2713-40b...|         Minus 8|4e78efc4-e545-47a...|      Elysian Fields|
|user_000001|ad0811ea-e213-451...|       Beanfield|fb51d2c4-cc69-412...|  Planetary Deadlock|
|user_000001|309e2dfc-678e-4d0...|        Dj Linus|4277434f-e3c2-41a

df: DataFrame = [user_id: string, artist_id: string ... 3 more fields]

The dataframe is then aggregated by user and track, to get the number of times a user has heard a particular track. 

In [9]:
val df_agg = df.select("user_id", "track_id")
            .groupBy("user_id", "track_id")
            .agg(count("*").alias("count")).orderBy("user_id")
val df_agg_filtered = df_agg.limit(5000)
df_agg_filtered.show()

show at cmd8.sc:5

19 / 19

show at cmd8.sc:5

200 / 200

show at cmd8.sc:5

1 / 1

+-----------+--------------------+-----+
|    user_id|            track_id|count|
+-----------+--------------------+-----+
|user_000001|20a5a368-3f4d-433...|   27|
|user_000001|d276b077-c05d-43c...|    4|
|user_000001|763b2ea5-3314-48c...|    2|
|user_000001|e7638bb7-bf57-435...|   14|
|user_000001|caca6626-7ba5-474...|    3|
|user_000001|19d1c947-fea8-459...|    2|
|user_000001|7b793966-abc8-423...|   10|
|user_000001|a494e993-a717-498...|    5|
|user_000001|687f53a1-b800-488...|    7|
|user_000001|d1b1d17a-87c8-410...|    4|
|user_000001|d9b7d831-e92a-4bb...|    1|
|user_000001|0024d72c-136f-49f...|    4|
|user_000001|95a65991-79d6-41d...|    4|
|user_000001|08cc9791-ac56-47d...|    6|
|user_000001|64e65892-1ab9-4c7...|   21|
|user_000001|97d2ae22-d794-49f...|   15|
|user_000001|f1400a93-16d0-452...|    3|
|user_000001|cd16ace9-2044-495...|    5|
|user_000001|c1680617-1a68-4ca...|   21|
|user_000001|d19e67ef-6ae5-446...|    3|
+-----------+--------------------+-----+
only showing top

df_agg: Dataset[Row] = [user_id: string, track_id: string ... 1 more field]
df_agg_filtered: Dataset[Row] = [user_id: string, track_id: string ... 1 more field]

In [10]:
val Array(training, test) = df_agg_filtered.randomSplit(Array[Double](0.8, 0.2), 18)

//revisit to make more efficient

val feat = df_agg_filtered.columns.filter(_ .contains("id"))
val inds = feat.map { colName =>
   new StringIndexer()
    .setInputCol(colName)
    .setOutputCol(colName.replace("id", "index"))
    .fit(df_agg_filtered)
    .setHandleInvalid("keep")
}

val va = new VectorAssembler()
    .setInputCols(Array("count"))
    .setOutputCol("count_assembled")

val scaler = new RobustScaler()
  .setInputCol("count_assembled")
  .setOutputCol("rating")

val pipeline = new Pipeline()
  .setStages(inds.toArray ++ Array(va, scaler))
  
val tr_s = pipeline.fit(training).transform(training)
val ts_s = pipeline.fit(training).transform(test)

val tr_full = tr_s.withColumn("rating_as_array", vector_to_array(tr_s("rating")).getItem(0))
val ts_full = ts_s.withColumn("rating_as_array", vector_to_array(ts_s("rating")).getItem(0))

val tr_final = tr_full.select("user_index", "track_index", "count","rating_as_array").orderBy("user_index")
val ts_final = ts_full.select("user_index", "track_index", "count", "rating_as_array").orderBy("user_index")

collect at StringIndexer.scala:204

19 / 19

collect at StringIndexer.scala:204

200 / 200

collect at StringIndexer.scala:204

...

mapPartitions at RobustScaler.scala:194

1 / 1

collect at RobustScaler.scala:166

1 / 1

training: Dataset[Row] = [user_id: string, track_id: string ... 1 more field]
test: Dataset[Row] = [user_id: string, track_id: string ... 1 more field]
feat: Array[String] = Array("user_id", "track_id")
inds: Array[org.apache.spark.ml.feature.StringIndexerModel] = Array(
  StringIndexerModel: uid=strIdx_e1557999ebf3, handleInvalid=keep,
  StringIndexerModel: uid=strIdx_30ab9236b359, handleInvalid=keep
)
va: VectorAssembler = VectorAssembler: uid=vecAssembler_fb5e10063edd, handleInvalid=error, numInputCols=1
scaler: RobustScaler = robustScal_826874f703df
pipeline: Pipeline = pipeline_151d3aeae50e
tr_s: DataFrame = [user_id: string, track_id: string ... 5 more fields]
ts_s: DataFrame = [user_id: string, track_id: string ... 5 more fields]
tr_full: DataFrame = [user_id: string, track_id: string ... 6 more fields]
ts_full: DataFrame = [user_id: string, track_id: string ... 6 more fields]
tr_final: Dataset[Row] = [user_index: double, track_index: double ... 2 more fields]
ts_final: Dataset[

In [11]:
tr_final.show()

show at cmd10.sc:1

19 / 19

show at cmd10.sc:1

200 / 200

show at cmd10.sc:1

1 / 1

+----------+-----------+-----+-------------------+
|user_index|track_index|count|    rating_as_array|
+----------+-----------+-----+-------------------+
|       0.0|       15.0|    7| 1.1666666666666665|
|       0.0|       71.0|   10| 1.6666666666666665|
|       0.0|       22.0|    2| 0.3333333333333333|
|       0.0|       23.0|    1|0.16666666666666666|
|       0.0|       24.0|    3|                0.5|
|       0.0|       25.0|   20|  3.333333333333333|
|       0.0|       26.0|    9|                1.5|
|       0.0|       29.0|    3|                0.5|
|       0.0|       34.0|    1|0.16666666666666666|
|       0.0|       36.0|    9|                1.5|
|       0.0|       38.0|    2| 0.3333333333333333|
|       0.0|       41.0|   21|                3.5|
|       0.0|       42.0|    7| 1.1666666666666665|
|       0.0|       43.0|   30|                5.0|
|       0.0|       45.0|   38|  6.333333333333333|
|       0.0|       53.0|    1|0.16666666666666666|
|       0.0|       60.0|    1|0

In [12]:
val als = new ALS()
  .setRank(5)
  .setUserCol("user_index")
  .setItemCol("track_index")
  .setRatingCol("rating_as_array")

val model = als.fit(tr_final)
model.setColdStartStrategy("drop")

val predictions = model.transform(ts_final)

val evaluator = new RegressionEvaluator()
  .setMetricName("rmse")
  .setLabelCol("rating_as_array")
  .setPredictionCol("prediction")

val rmse = evaluator.evaluate(predictions)
println(s"Root-mean-square error = $rmse")


rdd at ALS.scala:697

19 / 19

rdd at ALS.scala:697

200 / 200

isEmpty at ALS.scala:947

...

200 / 200

treeAggregate at Statistics.scala:58

13 / 13

Root-mean-square error = 0.7758869511290968


als: ALS = als_2229581d4195
model: org.apache.spark.ml.recommendation.ALSModel = ALSModel: uid=als_2229581d4195, rank=5
res11_2: org.apache.spark.ml.recommendation.ALSModel = ALSModel: uid=als_2229581d4195, rank=5
predictions: DataFrame = [user_index: double, track_index: double ... 3 more fields]
evaluator: RegressionEvaluator = RegressionEvaluator: uid=regEval_1dad03d350ff, metricName=rmse, throughOrigin=false
rmse: Double = 0.7758869511290968

In [13]:
val userRecs = model.recommendForAllUsers(10)
val movieRecs = model.recommendForAllItems(10)

userRecs: DataFrame = [user_index: int, recommendations: array<struct<track_index:int,rating:float>>]
movieRecs: DataFrame = [track_index: int, recommendations: array<struct<user_index:int,rating:float>>]

In [14]:
val firsttworecs = userRecs
    .withColumn("columns",expr("struct(recommendations[0] as rec1, recommendations[0] as rec2) as columns"))
    .select("user_index","columns.*")
firsttworecs.show()

show at cmd13.sc:4

100 / 100

show at cmd13.sc:4

1 / 1

show at cmd13.sc:4

...

100 / 100

show at cmd13.sc:4

75 / 75

+----------+-----------------+-----------------+
|user_index|             rec1|             rec2|
+----------+-----------------+-----------------+
|         1|[3444, 10.798533]|[3444, 10.798533]|
|         0| [4761, 18.36884]| [4761, 18.36884]|
+----------+-----------------+-----------------+



firsttworecs: DataFrame = [user_index: int, rec1: struct<track_index: int, rating: float> ... 1 more field]

In [15]:
val firstRec = firsttworecs.select("user_index", "rec1.*")
firstRec.show()

show at cmd14.sc:2

100 / 100

show at cmd14.sc:2

1 / 1

show at cmd14.sc:2

...

100 / 100

show at cmd14.sc:2

75 / 75

+----------+-----------+---------+
|user_index|track_index|   rating|
+----------+-----------+---------+
|         1|       3444|10.798533|
|         0|       4761| 18.36884|
+----------+-----------+---------+



firstRec: DataFrame = [user_index: int, track_index: int ... 1 more field]

In [16]:
// TODO make more efficient 
import spark.implicits._
val users = df_agg_filtered.select("user_id").map(_.getString(0)).distinct().collectAsList().toArray().map(_.asInstanceOf[String])
val usermap = new IndexToString()
    .setInputCol("user_index")
    .setOutputCol("user_id")
    .setLabels(users)

val userout = usermap.transform(firstRec)

val tracks = df_agg_filtered.select("track_id").map(_.getString(0)).distinct().collectAsList().toArray().map(_.asInstanceOf[String])
val trackmap = new IndexToString()
    .setInputCol("track_index")
    .setOutputCol("track_id")
    .setLabels(tracks)

val trackout = trackmap.transform(userout)

collectAsList at cmd15.sc:2

19 / 19

collectAsList at cmd15.sc:2

200 / 200

collectAsList at cmd15.sc:2

...

collectAsList at cmd15.sc:10

200 / 200

collectAsList at cmd15.sc:10

1 / 1

import spark.implicits._

users: Array[String] = Array("user_000001", "user_000002")
usermap: IndexToString = idxToStr_9da2bc993488
userout: DataFrame = [user_index: int, track_index: int ... 2 more fields]
tracks: Array[String] = Array(
  "20a5a368-3f4d-433b-897c-aa58590de72e",
  "d276b077-c05d-43c8-b168-63f12a6001b2",
  "763b2ea5-3314-48c6-88de-c1156b7d4c83",
  "e7638bb7-bf57-4352-ad29-6b40f2735514",
  "caca6626-7ba5-4748-be05-a9d49aa72287",
  "19d1c947-fea8-459a-91cb-6bab92903937",
  "7b793966-abc8-423d-bddc-1a761213d5f7",
  "a494e993-a717-498a-9dc3-c70ce26afe1d",
  "687f53a1-b800-4887-aba9-523b529c9e03",
  "d1b1d17a-87c8-410a-8d4b-345a7f127c88",
  "d9b7d831-e92a-4bbc-99d0-20aea8a4d4b6",
  "0024d72c-136f-49f2-9078-ce4b39b94d3f",
  "95a65991-79d6-41d1-811e-46937efb453b",
  "08cc9791-ac56-47d6-ab99-62bb97370dca",
  "64e65892-1ab9-4c7c-a9df-18bee9a1015f",
  "97d2ae22-d794-49f5-be1a-c97280822d72",
  "f1400a93-16d0-4525-a997-65dba8ce1057",
  "cd16ace9-2044-4959-bb15-9a2f820760b9",
  "c16

In [17]:
trackout.show()

show at cmd16.sc:1

100 / 100

show at cmd16.sc:1

1 / 1

show at cmd16.sc:1

...

show at cmd16.sc:1

100 / 100

show at cmd16.sc:1

75 / 75

+----------+-----------+---------+-----------+--------------------+
|user_index|track_index|   rating|    user_id|            track_id|
+----------+-----------+---------+-----------+--------------------+
|         1|       3444|10.798533|user_000002|581cb3a8-e229-400...|
|         0|       4761| 18.36884|user_000001|c45e44f5-9afb-44a...|
+----------+-----------+---------+-----------+--------------------+



In [18]:
val final_result = trackout.as("results")
        .join(df.as("in"), $"results.track_id" === $"in.track_id")
        .select("results.user_id", "in.track_name", "results.rating").distinct()
final_result.show()

show at cmd17.sc:4

100 / 100

show at cmd17.sc:4

19 / 19

show at cmd17.sc:4

...

100 / 100

show at cmd17.sc:4

75 / 75

+-----------+----------------+---------+
|    user_id|      track_name|   rating|
+-----------+----------------+---------+
|user_000001|I Fought The Law| 18.36884|
|user_000002|         Florida|10.798533|
+-----------+----------------+---------+



final_result: Dataset[Row] = [user_id: string, track_name: string ... 1 more field]